In [22]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [23]:
!pip install -q -U transformers datasets accelerate peft bitsandbytes trl

In [24]:
!pip install -U bitsandbytes
!pip install -q -U trl transformers accelerate peft bitsandbytes

In [25]:
!pip install trl

import and config

In [26]:
import torch
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, TrainingArguments
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training
from trl import SFTTrainer, SFTConfig
import os



In [27]:
!pip show trl transformers accelerate peft bitsandbytes

Name: trl
Version: 0.22.2
Summary: Train transformer language models with reinforcement learning.
Home-page: https://github.com/huggingface/trl
Author: Leandro von Werra
Author-email: leandro.vonwerra@gmail.com
License: 
Location: /usr/local/lib/python3.12/dist-packages
Requires: accelerate, datasets, transformers
Required-by: 
---
Name: transformers
Version: 4.56.1
Summary: State-of-the-art Machine Learning for JAX, PyTorch and TensorFlow
Home-page: https://github.com/huggingface/transformers
Author: The Hugging Face team (past and future) with the help of all our contributors (https://github.com/huggingface/transformers/graphs/contributors)
Author-email: transformers@huggingface.co
License: Apache 2.0 License
Location: /usr/local/lib/python3.12/dist-packages
Requires: filelock, huggingface-hub, numpy, packaging, pyyaml, regex, requests, safetensors, tokenizers, tqdm
Required-by: peft, sentence-transformers, trl
---
Name: accelerate
Version: 1.10.1
Summary: Accelerate
Home-page: https

In [28]:
!pip show trl

Name: trl
Version: 0.22.2
Summary: Train transformer language models with reinforcement learning.
Home-page: https://github.com/huggingface/trl
Author: Leandro von Werra
Author-email: leandro.vonwerra@gmail.com
License: 
Location: /usr/local/lib/python3.12/dist-packages
Requires: accelerate, datasets, transformers
Required-by: 


In [29]:
print(dataset[0])

{'instruction': 'Generate a Jest unit test for the following React component using React Testing Library.\n\nFollow these critical best practices for writing effective tests and avoid common mistakes:\n\n* **Query by User-Facing Attributes:** Prioritize querying elements the way a user would find them. Use roles, labels, and text (`getByRole`, `getByLabelText`, `getByText`). Avoid implementation details like class names or relying on `data-testid` as a first resort.\n* **Use `screen` for Queries:** Always use the `screen` object for querying the DOM (`screen.getByRole(...)`). Do not destructure query methods from `render`.\n* **Prefer `user-event` over `fireEvent`:** Use `@testing-library/user-event` for simulating user interactions as it more closely resembles real browser behavior.\n* **Use `*By` for Presence, `queryBy` for Absence:**\n    * Use `getBy...` or `getAllBy...` to assert an element is present (it throws an error if not found).\n    * Use `queryBy...` or `queryAllBy...` to

2. Paths & Model Names

In [30]:
model_name = "deepseek-ai/deepseek-coder-v2-lite-instruct"
dataset_path = "/content/drive/MyDrive/Fine-tuning/dataset.json"
output_dir = "/content/drive/MyDrive/Fine-tuning/outputs"
final_model_dir = "/content/drive/MyDrive/Fine-tuning/final_model"

3. Load Dataset

In [31]:
dataset = load_dataset("json", data_files=dataset_path, split="train")

# Debug check: print one row
print(dataset[0])

{'instruction': 'Generate a Jest unit test for the following React component using React Testing Library.\n\nFollow these critical best practices for writing effective tests and avoid common mistakes:\n\n* **Query by User-Facing Attributes:** Prioritize querying elements the way a user would find them. Use roles, labels, and text (`getByRole`, `getByLabelText`, `getByText`). Avoid implementation details like class names or relying on `data-testid` as a first resort.\n* **Use `screen` for Queries:** Always use the `screen` object for querying the DOM (`screen.getByRole(...)`). Do not destructure query methods from `render`.\n* **Prefer `user-event` over `fireEvent`:** Use `@testing-library/user-event` for simulating user interactions as it more closely resembles real browser behavior.\n* **Use `*By` for Presence, `queryBy` for Absence:**\n    * Use `getBy...` or `getAllBy...` to assert an element is present (it throws an error if not found).\n    * Use `queryBy...` or `queryAllBy...` to

4. Model & Tokenizer

In [32]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    torch_dtype=torch.bfloat16,
    device_map="auto",
    trust_remote_code=True,
)

tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

5. LoRA Config

In [33]:
model = prepare_model_for_kbit_training(model)

peft_config = LoraConfig(
    r=8,
    lora_alpha=16,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, peft_config)

6. Training Config

In [34]:
training_args = SFTConfig(
    output_dir=output_dir,
    num_train_epochs=2,
    per_device_train_batch_size=1,
    gradient_accumulation_steps=2,
    learning_rate=2e-4,
    logging_steps=10,
    save_steps=50,
    fp16=True,
    push_to_hub=False,
    dataset_text_field=None
)

In [37]:
print("Test formatting func output:")
print(formatting_prompts_func(dataset[0]))

Test formatting func output:
<｜begin▁of▁sentence｜>User: Generate a Jest unit test for the following React component using React Testing Library.

Follow these critical best practices for writing effective tests and avoid common mistakes:

* **Query by User-Facing Attributes:** Prioritize querying elements the way a user would find them. Use roles, labels, and text (`getByRole`, `getByLabelText`, `getByText`). Avoid implementation details like class names or relying on `data-testid` as a first resort.
* **Use `screen` for Queries:** Always use the `screen` object for querying the DOM (`screen.getByRole(...)`). Do not destructure query methods from `render`.
* **Prefer `user-event` over `fireEvent`:** Use `@testing-library/user-event` for simulating user interactions as it more closely resembles real browser behavior.
* **Use `*By` for Presence, `queryBy` for Absence:**
    * Use `getBy...` or `getAllBy...` to assert an element is present (it throws an error if not found).
    * Use `que

7. Formatting Function

In [52]:
def formatting_prompts_func(example):
    messages = [
        {"role": "user", "content": example["instruction"]},
        {"role": "assistant", "content": example["output"]},
    ]
    text = tokenizer.apply_chat_template(
        messages, tokenize=False, add_generation_prompt=False
    )
    return text

In [53]:
print(formatting_prompts_func(dataset[0]))

<｜begin▁of▁sentence｜>User: Generate a Jest unit test for the following React component using React Testing Library.

Follow these critical best practices for writing effective tests and avoid common mistakes:

* **Query by User-Facing Attributes:** Prioritize querying elements the way a user would find them. Use roles, labels, and text (`getByRole`, `getByLabelText`, `getByText`). Avoid implementation details like class names or relying on `data-testid` as a first resort.
* **Use `screen` for Queries:** Always use the `screen` object for querying the DOM (`screen.getByRole(...)`). Do not destructure query methods from `render`.
* **Prefer `user-event` over `fireEvent`:** Use `@testing-library/user-event` for simulating user interactions as it more closely resembles real browser behavior.
* **Use `*By` for Presence, `queryBy` for Absence:**
    * Use `getBy...` or `getAllBy...` to assert an element is present (it throws an error if not found).
    * Use `queryBy...` or `queryAllBy...` t

8. Trainer

In [54]:
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    peft_config=peft_config,
    args=training_args,
    formatting_func=formatting_prompts_func,
)

print("🚀 Starting training...")
trainer.train()
print("✅ Training finished!")

`rope_scaling`'s factor field must be a float >= 1, got 40
`rope_scaling`'s beta_fast field must be a float, got 32
`rope_scaling`'s beta_slow field must be a float, got 1
/usr/local/lib/python3.12/dist-packages/peft/tuners/lora/bnb.py:348: UserWarning: Merge lora module to 4-bit linear may get different generations due to rounding errors.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/peft/tuners/tuners_utils.py:196: UserWarning: Already found a `peft_config` attribute in the model. This will lead to having multiple adapters in the model. Make sure to know what you are doing!
  warnings.warn(


Applying formatting function to train dataset:   0%|          | 0/11 [00:00<?, ? examples/s]

Adding EOS to train dataset:   0%|          | 0/11 [00:00<?, ? examples/s]

Tokenizing train dataset:   0%|          | 0/11 [00:00<?, ? examples/s]

KeyError: None

9. Merge LoRA & Save

In [ ]:
from peft import AutoPeftModelForCausalLM

print("🧠 Merging LoRA adapter and saving final model...")
del model
torch.cuda.empty_cache()

model = AutoPeftModelForCausalLM.from_pretrained(
    training_args.output_dir,
    low_cpu_mem_usage=True,
    torch_dtype=torch.float16,
    device_map="auto"
)
merged_model = model.merge_and_unload()
merged_model.save_pretrained(final_model_dir, safe_serialization=True)
tokenizer.save_pretrained(final_model_dir)
print(f"✅ Final model saved to: {final_model_dir}")